# Model Comparisons

Compare with:

+ Arc
+ Act 1 & 2
+ Demo (Pulsed and SS)

// also adds proto reactor design

## 0.0 Bootup Notebook

In [1]:
rmprocs(collect(2:Sys.CPU_CORES))
addprocs(Sys.CPU_CORES-1)

@everywhere using Fussy
Main.IJulia.set_current_module(Fussy)

return

## 1.0 Set Defaults

In [67]:
cur_points = 41
min_temp = 0
max_temp = 40

cur_temps = linspace(min_temp, max_temp, cur_points)

cur_decks = [:arc, :act_1, :act_2, :demo_pulsed, :demo_steady, :scylla, :charybdis]

return

## 2.0 Define Make Function

In [3]:
function make_scans()
    tmp_scans = Dict()

    for cur_deck in cur_decks
        println(cur_deck)

        cur_symbol_pulsed = cur_deck
        cur_symbol_simple = Symbol("$(cur_deck)_simple")
        cur_symbol_steady = Symbol("$(cur_deck)_steady")

        tmp_scans[cur_symbol_pulsed] = Scan(cur_temps, deck=cur_deck, is_pulsed=true, is_consistent=false)
        tmp_scans[cur_symbol_simple] = Scan(cur_temps, deck=cur_deck, is_pulsed=false, is_consistent=false)
        tmp_scans[cur_symbol_steady] = Scan(cur_temps, deck=cur_deck, is_pulsed=false, is_consistent=true)

        sleep(0.25)
    end

    println("done.")
        
    tmp_scans
end

return

## 3.0 Get Data

In [4]:
cur_solutions = Dict()

for cur_deck in cur_decks
    tmp_symbol = Symbol("$(cur_deck)_solution")
    isdefined(Fussy, tmp_symbol) || continue
    cur_solutions[cur_deck] = getfield(Fussy, tmp_symbol)()
end

return

In [5]:
using FileIO
using JLD2

In [6]:
has_scans = isdefined(Fussy, :cur_scans) && !isempty(cur_scans)

cur_file = "../data/comparisons.jld2"

if !has_scans && isfile(cur_file)
    has_scans = true
    try
        cur_scans = load(cur_file, "cur_scans")
    catch
        has_scans = false
    end
end

if !has_scans
    cur_scans = make_scans()
    save(cur_file, Dict("cur_scans" => cur_scans))
end 

return

act_1


Progress: 100%|█████████████████████████████████████████| Time: 0:01:46
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
Progress: 100%|█████████████████████████████████████████| Time: 0:01:23


act_2


Progress: 100%|█████████████████████████████████████████| Time: 0:01:04
Progress: 100%|█████████████████████████████████████████| Time: 0:01:06


demo_steady


...

charybdis


Progress: 100%|█████████████████████████████████████████| Time: 0:01:07
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
Progress: 100%|█████████████████████████████████████████| Time: 0:01:24


arc


Progress: 100%|█████████████████████████████████████████| Time: 0:01:04
Progress: 100%|█████████████████████████████████████████| Time: 0:01:17


done.


## 4.0 Prep Plots

In [12]:
cur_fields = [ 
    "T_bar", "n_bar", "I_P", "R_0", "B_0", "R_CS",
    "P_F", "f_IN", "f_BS", "f_CD", "W_M", "cost", "eta_CD",
    "norm_beta_N", "norm_q_95", "norm_P_E", "norm_P_W", "b", "c", "d"
]

sort!(cur_fields)

cur_xvars = deepcopy(cur_fields)
cur_yvars = deepcopy(cur_fields)

filter!(cur_var -> cur_var != "B_0", cur_xvars)
filter!(cur_var -> cur_var != "R_0", cur_yvars)

unshift!(cur_xvars, "B_0")
unshift!(cur_yvars, "R_0")

return

In [13]:
using Plots
# pgfplots()

plotlyjs()

Plots.PGFPlotsBackend()

In [14]:
using Interact
using LaTeXStrings
using StringCases

## 5.0 Make Plots

In [81]:
@manipulate for xvar in cur_xvars, yvar in cur_yvars, deck in sort(cur_decks), showbad=[false,true]
    plot()
    
    cur_modes = Dict(
        "" => :solid
    )
    
    if startswith(string(deck), "demo")
        cur_modes["_simple"] = :dot
    else
        cur_modes["_steady"] = :dot
    end
    
    max_x = 0
    max_y = 0
    
    for (cur_super_key, cur_super_value) in cur_modes
        
        ( cur_super_key != "_steady" && ( xvar == "eta_CD" || yvar == "eta_CD" ) ) && continue
        
        tmp_scan = cur_scans[Symbol("$(deck)$(cur_super_key)")]

        tmp_dict = Dict(
            "beta" => tmp_scan.beta_reactors,
            "kink" => tmp_scan.kink_reactors,
            "wall" => tmp_scan.wall_reactors
        )

        for (cur_key, cur_value) in tmp_dict
            max_branches = maximum(map(tmp_reactor -> tmp_reactor.branch_id, cur_value))

            for cur_branch in 1:max_branches
                tmp_reactors = filter(tmp_value -> tmp_value.branch_id == cur_branch, cur_value)

                filter!(tmp_reactor -> tmp_reactor.B_0 < 40, tmp_reactors)
                filter!(tmp_reactor -> tmp_reactor.R_0 < 40, tmp_reactors)
                
                tmp_v = map(tmp_reactor -> tmp_reactor.is_valid, tmp_reactors)
                ( ( showbad ? length(tmp_reactors) : count(tmp_v) ) > 2 ) || continue

                tmp_x = map(tmp_reactor -> getfield(tmp_reactor, Symbol(xvar)), tmp_reactors)
                tmp_y = map(tmp_reactor -> getfield(tmp_reactor, Symbol(yvar)), tmp_reactors)

                tmp_t = map(tmp_reactor -> tmp_reactor.T_bar, tmp_reactors)
                tmp_i = map(tmp_reactor -> tmp_reactor.I_P, tmp_reactors)

                sort_lists!(tmp_t, tmp_v, tmp_x, tmp_y, tmp_i)

                if showbad
                    first_index = 1
                    last_index = length(tmp_reactors)
                else
                    first_index = minimum(find(tmp_v))
                    last_index = maximum(find(tmp_v))
                end


                ( first_index == 1 ) || ( first_index -= 1 )
                ( last_index == length(tmp_reactors) ) || ( last_index += 1 )
                
                tmp_x = tmp_x[first_index:last_index]
                tmp_y = tmp_y[first_index:last_index]
                
                tmp_v = tmp_v[first_index:last_index]
                
                max_x = max(max_x, maximum(tmp_x))
                max_y = max(max_y, maximum(tmp_y))

                cur_prefix = cur_super_key == "" ? "Pulsed" : ( cur_super_key == "_steady" ) ? "Steady" : "Simple"
                cur_label = "$(cur_prefix) -- $(capitalize(cur_key))"
                ( max_branches > 1 ) && ( cur_label *= " ($(romanize(cur_branch)))" )
                
                plot!(tmp_x, tmp_y, label=cur_label, style=cur_super_value)
            end
        end
        
    end
    
    min_x = 0
    min_y = 0
    
    max_x *= 1.4
    max_y *= 1.4
    
    ( xvar == "P_F" ) || 
        ( max_x = min(30, max_x) )
    
    ( yvar == "P_F" ) || 
        ( max_y = min(30, max_y) )
    
    ( xvar == "W_M" ) && ( max_x = 150 )
    ( yvar == "W_M" ) && ( max_y = 150 )
    
    ( xvar == "R_0" || xvar == "B_0" ) && ( max_x = 30 )
    ( yvar == "R_0" || yvar == "B_0" ) && ( max_y = 30 )
    
    if startswith(xvar, "norm_")
        max_x = 2.0 
    elseif startswith(xvar, "f_")
        max_x = 1.0
    elseif xvar == "cost"
        max_x = 0.1
    end
    
    if startswith(yvar, "norm_")
        max_y = 2.0 
    elseif startswith(yvar, "f_")
        max_y = 1.0
    elseif yvar == "cost"
        max_y = 0.1
    end
    
    if haskey(cur_solutions, deck)
        cur_soln = cur_solutions[deck]
        cur_soln_x = getfield(cur_soln, Symbol(xvar))
        cur_soln_y = getfield(cur_soln, Symbol(yvar))
        
        ( cur_soln_x != nothing ) &&
            plot!([cur_soln_x, cur_soln_x], [min_y, max_y], color=:black, opacity=0.3, label="", style=:dash)
        
        ( cur_soln_y != nothing ) &&
            plot!([min_x, max_x], [cur_soln_y, cur_soln_y], color=:black, opacity=0.3, label="", style=:dash)
        
        ( cur_soln_x != nothing && cur_soln_y != nothing ) &&
            scatter!([cur_soln_x], [cur_soln_y], color=:black, opacity=0.3, label="")
        
        plot!([0,0], [0,0], color=:black, opacity=0.3, label="Model")
    end
    
    xlims!(min_x, max_x)
    ylims!(min_y, max_y)
    
    xname = xvar
    yname = yvar

    xlabel!(xname)
    ylabel!(yname)
    
    xname = replace(xname, "eta_CD", "\\eta_{CD}")
    yname = replace(yname, "eta_CD", "\\eta_{CD}")
    
    xname = replace(xname, "95", "{95}")
    yname = replace(yname, "95", "{95}")

    xname = replace(xname, "beta", "\\beta")
    yname = replace(yname, "beta", "\\beta")
    
    endswith(xname, "_bar") && ( xname = "\\overline $(split(xname, "_")[1])" )
    endswith(yname, "_bar") && ( yname = "\\overline $(split(yname, "_")[1])" )
    
    startswith(xname, "norm_") && ( xname = "\\left( $(replace(xname, "norm_", "")) \\right)_{norm}" )
    startswith(yname, "norm_") && ( yname = "\\left( $(replace(yname, "norm_", "")) \\right)_{norm}" )
    
    startswith(xname, "f_") && ( xname = join(map(cur_string -> "{$(cur_string)}", split(xname, "_")), "_") )
    startswith(yname, "f_") && ( yname = join(map(cur_string -> "{$(cur_string)}", split(yname, "_")), "_") )
    
#     xlabel!(latexstring(xname))
#     ylabel!(latexstring(yname))
    
    cur_title = join(map(capitalize, split(string(deck), "_")), " ")
    title!(cur_title)
    
    plot!()
end

Interact.Options{:ToggleButtons,String}(477: "input-193" = B_0 String , "xvar", "B_0", "B_0", 1, Interact.OptionDict(DataStructures.OrderedDict("B_0"=>"B_0","I_P"=>"I_P","P_F"=>"P_F","R_0"=>"R_0","R_CS"=>"R_CS","T_bar"=>"T_bar","W_M"=>"W_M","b"=>"b","c"=>"c","cost"=>"cost"…), Dict("W_M"=>"W_M","I_P"=>"I_P","c"=>"c","B_0"=>"B_0","R_0"=>"R_0","b"=>"b","n_bar"=>"n_bar","eta_CD"=>"eta_CD","f_IN"=>"f_IN","norm_P_W"=>"norm_P_W"…)), Any[], Any[], true, "horizontal", true)

Interact.Options{:ToggleButtons,String}(479: "input-194" = R_0 String , "yvar", "R_0", "R_0", 1, Interact.OptionDict(DataStructures.OrderedDict("R_0"=>"R_0","B_0"=>"B_0","I_P"=>"I_P","P_F"=>"P_F","R_CS"=>"R_CS","T_bar"=>"T_bar","W_M"=>"W_M","b"=>"b","c"=>"c","cost"=>"cost"…), Dict("W_M"=>"W_M","I_P"=>"I_P","c"=>"c","R_0"=>"R_0","B_0"=>"B_0","b"=>"b","n_bar"=>"n_bar","eta_CD"=>"eta_CD","f_IN"=>"f_IN","norm_P_W"=>"norm_P_W"…)), Any[], Any[], true, "horizontal", true)

Interact.Options{:ToggleButtons,Symbol}(481: "input-195" = act_1 Symbol , "deck", :act_1, "act_1", 1, Interact.OptionDict(DataStructures.OrderedDict("act_1"=>:act_1,"act_2"=>:act_2,"arc"=>:arc,"charybdis"=>:charybdis,"demo_pulsed"=>:demo_pulsed,"demo_steady"=>:demo_steady,"scylla"=>:scylla), Dict(:act_1=>"act_1",:scylla=>"scylla",:demo_steady=>"demo_steady",:charybdis=>"charybdis",:act_2=>"act_2",:arc=>"arc",:demo_pulsed=>"demo_pulsed")), Any[], Any[], true, "horizontal", true)

Interact.Options{:ToggleButtons,Bool}(483: "input-196" = false Bool , "showbad", false, "false", 1, Interact.OptionDict(DataStructures.OrderedDict("false"=>false,"true"=>true), Dict(false=>"false",true=>"true")), Any[], Any[], true, "horizontal", true)